In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import math

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

train_log_dir = './logs'
if not tf.gfile.Exists(train_log_dir):
    tf.gfile.MakeDirs(train_log_dir)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# initialize data

images = mnist.train.images
labels = mnist.train.labels
tlabels = tf.convert_to_tensor(labels,name="labels")

images = images.reshape(images.shape[0], 28,28, 1)

num_datapoints = images.shape[0]

validation_images = mnist.validation.images
validation_images = validation_images.reshape(
    validation_images.shape[0], 28, 28, 1
)
validation_labels = mnist.validation.labels


In [ ]:
def run_net(images):
    net = slim.layers.conv2d(images, 32, [5,5], scope='conv1')
    net = slim.layers.max_pool2d(net, [2,2], scope='pool1')
    net = slim.layers.conv2d(net, 64, [5,5], scope='conv2')
    net = slim.layers.max_pool2d(net, [2,2], scope='pool2')
    net = slim.layers.flatten(net, scope='flatten3')
    net = slim.layers.fully_connected(net, 1024, scope='fully_connected4')
    net = slim.layers.fully_connected(net, 10, activation_fn=None, scope='fully_connected5')
    return net

x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y = tf.placeholder(tf.float32, shape=[None, 10])
predictions = run_net(x)
total_loss = tf.nn.softmax_cross_entropy_with_logits(
    logits=predictions,labels=y
)

correct = tf.equal(tf.argmax(predictions,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

learn_rate = 0.001
train_step = tf.train.AdamOptimizer(learn_rate).minimize(total_loss)

In [ ]:
batch_size = 50
sess = tf.Session()
sess.run(tf.global_variables_initializer())

epochs = 10
iterations = math.ceil(epochs * num_datapoints / batch_size)
print (iterations)

for i in range(iterations):
    batch = mnist.train.next_batch(batch_size)
    batch_images = batch[0].reshape(batch_size, 28, 28, 1).tolist()
    batch_labels = batch[1]

    sess.run(train_step, feed_dict={x: batch_images, y:batch_labels})

    if i%1000 == 0:
        train_accuracy = sess.run(
            accuracy,
            feed_dict={x: validation_images, y:validation_labels}
        )
        print("step %d, training accuracy %g"%(i, train_accuracy))


11000
step 0, training accuracy 0.2034
step 1000, training accuracy 0.9846
step 2000, training accuracy 0.9892
step 3000, training accuracy 0.9896
step 4000, training accuracy 0.991
step 5000, training accuracy 0.9902


In [ ]:
# evalutate test data

test_images = mnist.test.images
test_images = test_images.reshape(test_images[0],28,28,1)
test_labels = mnist.text.labels

test_accuracy = sess.run(
    accuracy,
    feed_dict={x: test_images, y:test_labels}
)